In [ ]:
pip install beautifulsoup4

In [ ]:
from bs4 import BeautifulSoup
import requests

def fetch_html_tables(url):
    "Returns a list of tables in the html of url"
    page = requests.get(url)
    bs=BeautifulSoup(page.content)
    tables=bs.find_all('table')
    return tables

tables=fetch_html_tables('https://www.vg.no/spesial/skattelister/2020/?fbclid=IwAR0AyqU-1UDa6IBqSKqS2IfNr0KFMewz8esKEV6YVlMCYtCxSa8wcD6hu3g')
table_html=tables[0]

#printing top
print(str(table_html)[:1000])

In [ ]:
def html_to_table(html):
    "Returns the table defined in html as a list"
    #defining the table:
    table=[]
    #iterating over all rows
    for row in html.find_all('tr'):
        r=[]
        #finding all cells in each row:
        cells=row.find_all('td')
        
        #if no cells are found, look for headings
        if len(cells)==0:
            cells=row.find_all('th')
            
        #iterate over cells:
        for cell in cells:
            cell=format(cell)
            r.append(cell)
        
        #append the row to t:
        table.append(r)
    return table

def format(cell):
    "Returns a string after converting bs4 object cell to clean text"
    if cell.content is None:
        s=cell.text
    elif len(cell.content)==0:
        return ''
    else:
        s=' '.join([str(c) for c in cell.content])
        
    #here you can add additional characters/strings you want to 
    #remove, change punctuations or format the string in other
    #ways:
    s=s.replace('\xa0','')
    s=s.replace('\n','')
    return s

table=html_to_table(table_html)

#printing top
print(str(table)[:1000])

In [ ]:
';'.join(table[0])

In [ ]:
def save_data(file_name,table):
    "Saves table to file_name"
    f=open(file_name,'w')
    for row in table:
        f.write(';'.join(row)+'\n')
    f.close()
    
save_data('skattelister.csv',table)

In [ ]:
import pandas as pd
df = pd.read_csv('skattelister.csv', delimiter=';', encoding='utf-8')

df

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
df = df.rename(columns={'Unnamed: 0': 'Nummer', 'Inntekt': 'Inntekt'}) #Inntektî 
#df.columns=df.columns.str.replace('[î]','', regex=True)
df["Formue"] = pd.to_numeric(df["Formue"])
df["Skatt"] = pd.to_numeric(df["Skatt"])
df["Inntekt"] = pd.to_numeric(df["Inntekt"])

In [ ]:
fig,ax=plt.subplots()

#adding axis lables:
ax.set_ylabel('Skatt')
ax.set_xlabel('Inntekt')

#plotting the function:
ax.scatter(df['Inntekt'], df['Skatt'])


In [ ]:
y=df['Skatt']
pd.DataFrame(y)

In [ ]:
x=pd.DataFrame(np.log(df['Inntekt']))
x['intercept']=1
x

In [14]:
#pip install --upgrade -- no deps statsmodels

#conda install statsmodels

#pip install patsy

import statsmodels

from statsmodels.regression.linear_model import OLS

res=OLS(y,x).fit()

print(res.summary())

ModuleNotFoundError: No module named 'statsmodels.regression'

In [13]:
res.params

NameError: name 'res' is not defined

In [ ]:
x=np.linspace(min(np.log(df['Inntekt'])), max(np.log(df['Inntekt'])), 100)

regression_line=res.params['intercept']+res.params['Inntekt']*x

ax.plot(x, regression_line,color='red')
fig

In [ ]:
#Her har jeg brukt koden du ga oss, og lagt inn min egen tabell fra vg sin side om skattelister. Har laget en
#regresjon. Vi ser at inntekten har en viss sammenheng på hva de betaler i skatt, men at for de rikeste i samfunnet
#så varierer det veldig hva de betaler i skatt sammenlignet med inntekten deres.

#Først skrapte jeg data fra nettsiden, så lagde jeg det om til en csv fil og så et dataframe.
#Lagde deretter en plot, hvor vi la til en regresjonslinje.
